# Save & Load

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Checkpoint

In [14]:
save_path = 'ckpt'

In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

## Training

http://localhost:6006

In [16]:
steps_per_epoch = len(train_paths) // batch_size
test_steps = len(test_paths) // batch_size

history = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=test_steps,
    epochs=num_epochs, 
    callbacks=[checkpoint]
)

Epoch 1/20
31/31 [==============================] - 7s 225ms/step - loss: 2.3246 - accuracy: 0.1074 - val_loss: 2.3011 - val_accuracy: 0.0897
Epoch 2/20
31/31 [==============================] - 7s 237ms/step - loss: 2.2967 - accuracy: 0.1105 - val_loss: 2.2950 - val_accuracy: 0.1018
Epoch 3/20
31/31 [==============================] - 7s 217ms/step - loss: 2.2627 - accuracy: 0.1684 - val_loss: 2.2701 - val_accuracy: 0.1310
Epoch 4/20
31/31 [==============================] - 7s 225ms/step - loss: 2.2158 - accuracy: 0.1498 - val_loss: 2.1847 - val_accuracy: 0.1875
Epoch 5/20
31/31 [==============================] - 7s 220ms/step - loss: 2.1566 - accuracy: 0.1839 - val_loss: 2.0968 - val_accuracy: 0.2147
Epoch 6/20
31/31 [==============================] - 7s 237ms/step - loss: 2.0596 - accuracy: 0.2407 - val_loss: 2.1248 - val_accuracy: 0.2097
Epoch 7/20
31/31 [==============================] - 7s 234ms/step - loss: 2.0416 - accuracy: 0.2510 - val_loss: 1.9973 - val_accuracy: 0.2450
Epoch 

# Saving Model

In [17]:
save_path = 'my_model.h5'

In [18]:
# Save the model architecture and the weights
model.save(save_path, include_optimizer=True)

In [19]:
# Load the model architecture and the weights
model = tf.keras.models.load_model('my_model.h5')

# Saving Model - 2

In [20]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [21]:
from tensorflow.keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [22]:
import h5py

model_file = h5py.File('my_model.h5','r+')    

In [23]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [24]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [25]:
model_file['model_weights']['conv2d'].keys()

<KeysViewHDF5 ['conv2d']>

In [26]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [27]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [28]:
# weights
np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

array([[[[-3.19316238e-02,  8.07693508e-03, -3.29100783e-03,
           1.19189858e-01,  3.14581580e-02, -1.23483710e-01,
           2.05476284e-02,  5.33935912e-02,  1.24092072e-01,
           1.15196660e-01, -1.70140028e-01, -1.57607794e-02,
          -3.80136333e-02,  9.84424427e-02, -5.84885739e-02,
          -1.27600893e-01,  5.46499155e-02,  2.31469050e-02,
          -1.25634953e-01, -1.33529097e-01,  3.81863490e-03,
           4.76853624e-02,  2.55253576e-02,  1.05632901e-01,
          -2.89070681e-02, -9.98184159e-02,  5.86258918e-02,
          -2.91876979e-02, -2.31441371e-02,  3.16969422e-03,
           1.19447283e-01,  1.50333002e-01],
         [ 8.67402181e-02, -1.87144801e-02, -8.13820511e-02,
           1.38858445e-02,  8.05762410e-02, -1.31813645e-01,
          -4.71420437e-02,  5.71946427e-02, -2.84033995e-02,
           2.82548014e-02,  1.75196044e-02, -2.32636109e-02,
          -1.33076832e-01,  9.06214342e-02,  7.74158612e-02,
          -1.03757801e-02,  1.05799697e-